# Assignment 2 - Build a simple binary search engine

1- Create Dictionary/Vocabulary.    
2- For each term in the dictionary, create a posting list for it    

## Dataset

[**RCV1 (Reuters Corpus Volume 1)**](https://www.jmlr.org/papers/volume5/lewis04a/lewis04a.pdf) is a widely used dataset in machine learning and natural language processing (NLP) research. It is a **collection of newswire stories** collected from the Reuters news agency in 1996. The RCV1 dataset contains approximately 800,000 manually categorized newswire stories from over 100 categories, such as politics, business, sports, and entertainment.   
For simplicity, we just use train subset of the dataset.

In [1]:
import pandas as pd

# Parse the text and extract the ID and terms for each document
with open('lyrl2004_vectors_train.dat', 'rb') as f:
    text = f.read().decode('utf-8')
documents = []

for doc in text.split('.I ')[1:]:
    doc_id, doc_text = doc.split('\n.W\n')
    terms = doc_text.split()
    documents.append({'ID': doc_id, 'Terms': ' '.join(terms)})

# Create a DataFrame with the ID and terms for each document
df = pd.DataFrame(documents)

In [2]:
df

,ID,Terms
0,2286,recov recov recov recov excit excit bring mexi...
1,2287,uruguay uruguay compan compan compan bring lim...
2,2288,spun stak compan compan compan compan compan c...
3,2289,spun stak compan compan compan compan compan c...
4,2290,compan compan limit planet planet planet plane...
...,...,...
23144,26145,ipi ipi ipi ipi ipi study market tight protest...
23145,26146,compan market launch ag target target saturday...
23146,26147,allot guent capac market saxon saxon saxon sax...
23147,26148,limit draw protest protest gener specif illeg ...


## Document-Term matrix
A *document-term matrix* is a **mathematical representation of text data**, where **rows represent individual terms**(words, phrases, or concepts) and **columns represent documents**. Each cell in the matrix contains the frequency or occurrence of a specific term in a particular document. The matrix can be binary, where each cell contains a 1 if the term is present in the document and a 0 if it is not, or it can be weighted, where the cell value is the frequency of the term in the document. The document-term matrix is a crucial component in many Natural Language Processing (NLP) and text mining tasks, such as topic modeling, sentiment analysis, and information retrieval.

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Build the document-term matrix using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Terms'])

<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


## Posting List
In computer science, a *posting list* is a database index storing a mapping from content, such as words or numbers, to its locations in a table, or in a document or a set of documents.

In [4]:
# Create a posting list for each term
terms = vectorizer.get_feature_names_out()
posting_lists = {}
for i, term in enumerate(terms):
    doc_indices = X[:, i].nonzero()[0]
    real_ids = [df.iloc[x].ID for x in doc_indices.tolist()]
    posting_lists[term] = real_ids

In [5]:
# Save the unique terms to a file
with open('unique_terms.txt', 'w') as f:
    f.write('\n'.join(terms))
    
# Save the posting lists to a file
with open('posting_lists.txt', 'w') as f:
    for term, posting_list in posting_lists.items():
        posting_str = ' '.join(str(doc_id) for doc_id in posting_list)
        f.write(f'{term}: {posting_str}\n')

## Binary Search
[Binary search](https://www.programiz.com/dsa/binary-search) is a **searching algorithm** for **finding an element’s position in a sorted list**. In this approach, the element is always searched in the middle of a portion of a list. Binary search can be implemented only on a sorted list of items.

In [6]:
def binary_search(search_term):
    # Open the posting_lists.txt file in read mode and read all the lines into a list
    with open('posting_lists.txt', 'r') as f:
        lines = f.readlines()
    
    # Initialize the indices for binary search
    left = 0
    right = len(lines) - 1
    
    # Loop until left index crosses the right index
    while left <= right:
        # Find the middle index of the current search range
        mid = (left + right) // 2
        # Extract the line corresponding to the middle index
        line = lines[mid]
        # Split the line into term and document IDs string using the colon separator
        term, document_ids = line.strip().split(': ')
        
        # Compare the search term with the current term
        if term == search_term:
            # If the search term matches the current term, return the list of document IDs as a list of strings
            return document_ids.split()
        elif term < search_term:
            # If the current term is smaller than the search term, update the left index to search in the right half
            left = mid + 1
        else:
            # If the current term is greater than the search term, update the right index to search in the left half
            right = mid - 1
            
    # If the search term is not found, return None
    return None



## Search term

In [7]:
def search():
    term = str(input('Search term...'))
    document_ids = binary_search(term)

    print(f'Searched term: {term}')
    # Check if the search term was found or not, and print the corresponding result
    if document_ids is not None:
        print(f'Document IDs: {document_ids}')
    else:
        print('Sorry, the given term not found!')

In [8]:
# forehead
search()

Searched term: forehead
Document IDs: ['14072', '16696', '16849', '24284', '24292']


In [9]:
# girlfriend
search()

Searched term: girlfriend
Document IDs: ['4163', '6240', '7592', '12105', '16679', '16888', '17988', '18099', '18209', '19765', '20383', '21133', '22477', '24514', '25253']


In [10]:
# apple
search()

Searched term: apple
Sorry, the given term not found!
